### This is an R2OpenBUGS program for the artificial example in Chapter 4, Section 4.5.

Model: Standard Structural Equation Model
Data Set Name: YO.dat
Sample Size: N=300

In [ ]:
setwd('/home/pal_bjartan/Backup/PhD/SEM-test-model/Lee2007')
source(".Rprofile")



#### (A) Codes for specification of the model using R language

In [126]:
model  <- function() {
    for(i in 1:N){
        #measurement equation model
        for(j in 1:P){
            y[i,j] ~ dnorm(mu[i,j],psi[j])
            ephat[i,j] <- y[i,j] - mu[i,j]
        }
        mu[i,1] <- eta[i]+alp[1]
        mu[i,2] <- lam[1] * eta[i] + alp[2]
        mu[i,3] <- lam[2] * eta[i] + alp[3]
        mu[i,4] <- xi[i,1] + alp[4]
        mu[i,5] <- lam[3] * xi[i,1] + alp[5]
        mu[i,6] <- lam[4] * xi[i,1] + alp[6]
        mu[i,7] <- xi[i,2] + alp[7]
        mu[i,8] <- lam[5] * xi[i,2] + alp[8]
        mu[i,9] <- lam[6] * xi[i,2] + alp[9]

        #structural equation model
        xi[i,1:2] ~ dmnorm(u[1:2], phi[1:2,1:2])
        eta[i] ~ dnorm(nu[i], psd)
        nu[i] <- gam[1] * xi[i,1] + gam[2] * xi[i,2]
        dthat[i] <- eta[i] - nu[i]
    } #end of i

    #priors on intercepts
    for(j in 1:9){alp[j]~dnorm(0.0, 1.0)}

    #priors on loadings and coefficients
    lam[1] ~ dnorm(0.8, psi[2])
    lam[2] ~ dnorm(0.8, psi[3])
    lam[3] ~ dnorm(0.8, psi[5])
    lam[4] ~ dnorm(0.8, psi[6])
    lam[5] ~ dnorm(0.8, psi[8])
    lam[6] ~ dnorm(0.8, psi[9])
    for(j in 1:2){ gam[j] ~ dnorm(0.5, psd) }
    
    #priors on precisions
    for(j in 1:P){
        psi[j] ~ dgamma(9.0, 4.0)
        sgm[j] <- 1/psi[j]
    }
    psd ~ dgamma(9.0, 4.0)
    sgd <- 1/psd
    phi[1:2,1:2] ~ dwish(R[1:2,1:2], 5)
    phx[1:2,1:2] <- inverse(phi[1:2, 1:2])

} #end of model

write.model(model, con = "./Chapter4/ch4-R2OpenBUGS-model.txt")


In [127]:
model <- paste0(getwd(), "/Chapter4/ch4-R2OpenBUGS-model.txt")
model

[1] "/home/pal_bjartan/Backup/PhD/SEM-test-model/Lee2007/Chapter4/ch4-R2OpenBUGS-model.txt"

#### (B) Check model

`bugs()` will autmatically check if model is syntactically correct. If
there are no errors, then the message *"model is syntactically correct"* will be printed in `log.txt` in the session's `tempdir()` or the directory defined in the `bugs()` function's `working.directory` argument.

#### (C) Load data

Read in dataset and save data object in the following format:

In [128]:
# Read in dataset as unnamed matrix
YO.dat <- read.csv("./Chapter4/ch4-WinBUGS-data.dat", header = FALSE, skip = 2)[,1:9] %>% 
as.matrix()  %>% 
unname()

# Save data as list in the following format
data <- list(
   N = 300,
   P = 9,
   u = c(0,0),
   y = YO.dat,
   R = structure(
      .Data=c( 1.0, 0.0,
               0.0, 1.0),
      .Dim = c(2,2))
)

The data will be loaded by `bugs()`' first argument `data`. When the data have been successfully loaded, the message *"data loaded"* should appear in the log file.

#### (D) Compile model

By default, `bugs()` simulate three chains of observations with three different sets of initial values (`n.chains = 3`). To change to a different number of Markov chains, add a different value to the `n.chains` argument. For this example the author used three chains, and the parameter can be left unchanged.

`bugs()` automatically compiles the model. If successful, the message "model
compiled" should appear in the log file.


#### (E) Load initial values

Initial values is stored as a function without input, which will be entered as `bugs()` second argument:

In [129]:
# Three different initial values
inits <- function() {
   list(alp = c(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
      lam = c(0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
      psi = c(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0),
      psd = 1.0,
      gam = c(1.0, 1.0),
      phi = structure(
         .Data = c(  0.2, 0.1,
                     0.1, 0.9),
         .Dim = c(2,2)
      )
   )

   list(alp = c(-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0),
      lam = c(-1.0, 0.0, 0.3, 0.6, 0.9, 1.0),
      psi = c(1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5),
      psd = 1.5,
      gam = c(-1.0,-1.0),
      phi = structure(
         .Data = c(  0.5, 0.2,
                     0.2, 0.6),
         .Dim = c(2,2)
      )
   )

   list(alp = c(2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0),
      lam = c(1.0, 0.3, 0.4, 0.5, 1.0, -1.0),
      psi = c(0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3),
      psd = 0.3,
      gam = c(2.0, 2.0),
      phi = structure(
         .Data = c(  0.9, 0.3,
                     0.3, 0.8),
         .Dim = c(2,2)
      )
   )
}

This is necessary to generate the nodes `xi`'s in the MCMC algorithm.
If the model is fully initialized after loading the initial values, then a message
saying *"initial values loaded: model initialized'* will appear in the log file. Otherwise, the message *"initial values loaded: model contains uninitialized nodes"* will appear.

#### (F) Run simulation and check convergence

##### **Step 1: Input the monitored parameters**

`bugs()` third argument, `parameters.to.save`, stores the parameters to be
monitored as a character vector:

In [154]:
param <- c("alp", "lam", "psi", "phi", "gam", "xi")

##### **Step 2: Sample observations for the monitored node**

`bugs()`fourth argument, `n.iter`, stores the total number of MCMC iterations (for example, 5000 or 10000; the default value is 2000).

In [ ]:
n.iter <- 5000

The argument `n.thin` sets the thinning rate (for example, k, i.e., the samples from every k-th iteration can be stored) into the box marked thin. The default is 1.

When the run finishes, the message `updates took ***s' will appear in the log file.

##### **Step 3: Check convergence**

The first few thousand iterations should be treated as burn-in iterations.
This is carried out by specifying the number in the `n.burnin` argument. The remaining iteration will be used to obtain the Bayesian statistics.

In [ ]:
n.burnin  <- 2000


Click trace in the Sample Monitor Tool window once. Plots of the simulated
observations against the number of iterations for all of the parameters in bb will
be shown for checking whether the algorithm has converged.

##### Step 4: Obtain Bayesian statistics

The results for statistics inference, for example, the mean (the Bayesian estimates), standard deviation, Monte Carlo error, will be given in the output of `bugs()` and in the log file.

To get estimates for omega_i (i.e. node xi), repeat the aforementioned steps 1-3
with additional iterations (for example, 4000 or 5000). Note that the output results are displayed
by a column vector (`xi[1,1],xi[1,2],xi[1,3],xi[2,1],xi[2,2],xi[2,3],......,xi[100,1],xi[100,2],xi[100,3]`).

To get the corresponding DIC values, set bugs()' argument `DIC = TRUE` (default).

In [156]:
bugs(
    data, 
    inits, 
    param, 
    model.file = model, 
    n.iter, 
    n.burnin = n.burnin, 
    saveExec = FALSE,
    codaPkg = FALSE,
    working.directory = paste0(getwd(), "/Chapter4/output")
)

Inference for Bugs model at "/home/pal_bjartan/Backup/PhD/SEM-test-model/Lee2007/Chapter4/ch4-R2OpenBUGS-model.txt", 
Current: 3 chains, each with 5000 iterations (first 2000 discarded)
Cumulative: n.sims = 9000 iterations saved
            mean   sd   2.5%    25%    50%    75%  97.5% Rhat n.eff
alp[1]       0.0  0.1   -0.1    0.0    0.0    0.1    0.1    1   130
alp[2]       0.0  0.1   -0.1    0.0    0.0    0.1    0.2    1   130
alp[3]       0.0  0.1   -0.1    0.0    0.0    0.0    0.1    1   210
alp[4]       0.1  0.1    0.0    0.1    0.1    0.2    0.3    1   330
alp[5]       0.1  0.1    0.0    0.1    0.1    0.2    0.2    1   320
alp[6]       0.0  0.1   -0.1    0.0    0.0    0.1    0.1    1   260
alp[7]       0.1  0.1    0.0    0.1    0.1    0.2    0.3    1   140
alp[8]       0.1  0.1   -0.1    0.0    0.1    0.1    0.2    1   160
alp[9]       0.0  0.1   -0.1    0.0    0.0    0.1    0.2    1   150
lam[1]       1.0  0.1    0.9    1.0    1.0    1.1    1.2    1  1200
lam[2]       0.7  0.0  